<a href="https://colab.research.google.com/github/Yekumo/video-captioning-deep-learning/blob/main/Video_Captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
print("PyTorch Version:", torch.__version__)
print("GPU Var mı?:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Adı:", torch.cuda.get_device_name(0))
else:
    print("GPU AKTİF DEĞİL! Ayarları kontrol et.")

PyTorch Version: 2.9.0+cu126
GPU Var mı?: True
GPU Adı: Tesla T4
